In [1]:
import pandas as pd
import os
from ast import literal_eval

### load df for all QAs

In [49]:
path_to_file_list = []
path = "temp"
for p in [p for p in os.listdir(path) if not p.startswith(".")]:
    path_to_file_inner_list = []
    for f in [f for f in os.listdir(f"{path}/{p}") if not f.startswith(".")]:
        path_to_file = f"{path}/{p}/{f}"
        # print(path_to_file)
        path_to_file_inner_list.append(path_to_file)
    path_to_file_list.append(path_to_file_inner_list)

In [51]:
# path_to_file_list [categorical, span-extraction, multiple-choice]
# cat_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[0]])
# 1 span-extraction with spans, 3 span-extraction old
span_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[2]])
# mcq_df = pd.concat([pd.read_csv(f) for f in path_to_file_list[4]])

### clean up for categorical QA

In [38]:
cat_df["logits_answer"] = cat_df["logits_answer"].str.replace("([\[\]])", "", regex=True)
cat_df["logits_answer"] = cat_df["logits_answer"].str.split()
cat_df["logits_answer_0"] = [logit[0] for logit in cat_df["logits_answer"]]
cat_df["logits_answer_1"] = [logit[1] for logit in cat_df["logits_answer"]]

In [39]:
cat_df.head()

,skill,reader,adapter,modelname,timestamp,answer,logits_answer,data_id,dataset,question,context,answer_dataset,logits_answer_0,logits_answer_1
0,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:08.066664,False,"[0.9633585, -0.6806862]",0,boolq,does ethanol take more energy make that produces,All biomass goes through at least some of thes...,False,0.9633585,-0.6806862
1,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:09.165009,True,"[0.11953807, 0.18089743]",1,boolq,is house tax and property tax are same,Property tax or 'house tax' is a local tax on ...,True,0.11953807,0.18089743
2,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:09.549410,True,"[-0.03293786, -0.00041977]",2,boolq,is pain experienced in a missing body part or ...,Phantom pain sensations are described as perce...,True,-0.03293786,-0.00041977
3,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:10.110081,True,"[0.07552637, 0.13251488]",3,boolq,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts is ...,True,0.07552637,0.13251488
4,categorical,bert-base-uncased,boolq,quant_onnx,2023-01-25 00:29:10.657743,False,"[0.9214445, -0.96274555]",4,boolq,is there a difference between hydroxyzine hcl ...,Hydroxyzine preparations require a doctor's pr...,True,0.9214445,-0.96274555


In [6]:
## TODO group 

In [65]:
df = pd.DataFrame(columns=[
    "skill", "reader", "adapter", "timestamp", 
    "answer_base", "logits_answer_base",
    "answer_quantized_model", "logits_answer_quantized_model", 
    "answer_onnx_model", "logits_answer_onnx_model",
    "answer_onnx_opt_model", "logits_answer_onnx_opt_model",
    "answer_quant_onnx_model", "logits_answer_quant_onnx_model",
    "answer_quant_onnx_opt_model", "logits_answer_quant_onnx_opt_model",
    "data_id", "dataset", "question", "context", "answer_dataset"
])

In [66]:
cat_df.keys()

Index(['skill', 'reader', 'adapter', 'modelname', 'timestamp', 'answer',
       'logits_answer', 'data_id', 'dataset', 'question', 'context',
       'answer_dataset', 'logits_answer_0', 'logits_answer_1'],
      dtype='object')

In [67]:
skill = cat_df["skill"].iloc[0]
skill

'categorical'

In [68]:
skill = cat_df["skill"].iloc[0]
for reader in cat_df["reader"].unique():
    # print(reader)
    reader_cat_df = cat_df[cat_df["reader"] == reader]
    # print(len(reader_cat_df["data_id"]))

    for data_id in reader_cat_df["data_id"].unique():
        s = reader_cat_df[reader_cat_df["data_id"] == data_id].reset_index()

        adapter = s["adapter"].iloc[0]
        data_set_name = adapter
        question = s["question"].iloc[0]
        context = s["context"].iloc[0]
        answer_dataset = s["answer_dataset"].iloc[0]

        interim = {}
        for model in s["modelname"].unique():
            # print(model)
            interim[model] = (
                s[s["modelname"] == model]["answer"].values[0],
                [
                    s[s["modelname"] == model]["logits_answer_0"].values[0],
                    s[s["modelname"] == model]["logits_answer_1"].values[0],
                ]
            )
            
        # print(interim)

        df.loc[len(df)] = [
            skill, reader, adapter,
            pd.Timestamp.now(),

            interim["base"][0], [interim["base"][1]],
            interim["quant_base"][0], [interim["quant_base"][1]],
            interim["onnx"][0], [logit for logit in interim["onnx"][1]],
            interim["onnx_opt"][0], [logit for logit in interim["onnx_opt"][1]],
            interim["quant_onnx"][0], [logit for logit in interim["quant_onnx"][1]],
            interim["quant_onnx_opt"][0], [logit for logit in interim["quant_onnx_opt"][1]],
                                                                                                                            
            data_id, data_set_name, question, context, answer_dataset

        ]

        # break

        

In [69]:
df

,skill,reader,adapter,timestamp,answer_base,logits_answer_base,answer_quantized_model,logits_answer_quantized_model,answer_onnx_model,logits_answer_onnx_model,...,logits_answer_onnx_opt_model,answer_quant_onnx_model,logits_answer_quant_onnx_model,answer_quant_onnx_opt_model,logits_answer_quant_onnx_opt_model,data_id,dataset,question,context,answer_dataset
0,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.813455,True,"[0.03632838, 0.1559807]",True,"[0.04307159, 0.14856294]",False,"[1.0141207, -0.70817924]",...,"[1.0141207, -0.70817924]",False,"[0.9633585, -0.6806862]",False,"[0.9633585, -0.6806862]",0,boolq,does ethanol take more energy make that produces,All biomass goes through at least some of thes...,False
1,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.821730,True,"[0.00471915, 0.4291983]",True,"[-0.00732881, 0.44615737]",True,"[-0.40855747, 0.8667419]",...,"[-0.40855747, 0.8667419]",True,"[0.11953807, 0.18089743]",True,"[0.11953807, 0.18089743]",1,boolq,is house tax and property tax are same,Property tax or 'house tax' is a local tax on ...,True
2,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.832132,True,"[-1.280071, 1.1713798]",True,"[-1.251884, 1.1318096]",True,"[-0.8122971, 0.83803236]",...,"[-0.8122971, 0.83803236]",True,"[-0.03293786, -0.00041977]",True,"[-0.03293786, -0.00041977]",2,boolq,is pain experienced in a missing body part or ...,Phantom pain sensations are described as perce...,True
3,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.843168,True,"[-1.1285974, 1.2845722]",True,"[-1.1000056, 1.2797506]",True,"[-0.94695413, 1.1307392]",...,"[-0.94695413, 1.1307392]",True,"[0.07552637, 0.13251488]",True,"[0.07552637, 0.13251488]",3,boolq,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts is ...,True
4,categorical,bert-base-uncased,boolq,2023-01-29 22:42:09.853807,False,"[1.1531763, -1.1785511]",False,"[1.2003236, -1.2420458]",False,"[1.3443897, -1.2073131]",...,"[1.3443897, -1.2073131]",False,"[0.9214445, -0.96274555]",False,"[0.9214445, -0.96274555]",4,boolq,is there a difference between hydroxyzine hcl ...,Hydroxyzine preparations require a doctor's pr...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,categorical,roberta-base,boolq,2023-01-29 22:43:14.480709,False,"[1.1614211, -1.1667186]",False,"[1.1709174, -1.1761547]",False,"[0.7405262, -0.75474274]",...,"[0.7405262, -0.75474274]",False,"[0.9792921, -0.9989573]",False,"[0.9792921, -0.9989573]",3265,boolq,is manic depression the same as bi polar,"Bipolar disorder, previously known as manic de...",True
6536,categorical,roberta-base,boolq,2023-01-29 22:43:14.492041,True,"[-2.1512992, 2.1035373]",True,"[-2.1589963, 2.1239989]",True,"[-1.7640444, 1.6682422]",...,"[-1.7640444, 1.6682422]",True,"[-1.5305727, 1.4393224]",True,"[-1.5305727, 1.4393224]",3266,boolq,was whiskey galore based on a true story,SS Politician was an 8000-ton cargo ship owned...,True
6537,categorical,roberta-base,boolq,2023-01-29 22:43:14.502872,True,"[-3.211123, 3.1748679]",True,"[-3.2806938, 3.2134044]",True,"[-2.742387, 2.6857572]",...,"[-2.742387, 2.6857572]",True,"[-2.6072588, 2.5575714]",True,"[-2.6072588, 2.5575714]",3267,boolq,are there plants on the international space st...,Plant research continued on the International ...,True
6538,categorical,roberta-base,boolq,2023-01-29 22:43:14.513952,True,"[-0.72820544, 0.7388733]",True,"[-0.7233548, 0.7610312]",False,"[0.39946824, -0.364812]",...,"[0.39946824, -0.364812]",False,"[0.13299496, -0.08860482]",False,"[0.13299496, -0.08860482]",3268,boolq,does the hockey puck have to cross the line to...,"In ice hockey, a goal is scored when the puck ...",True


In [ ]:
# df = pd.read_csv("in.csv",converters={"Col3": lambda x: x.strip("[]").split(", ")})

In [70]:
df.to_csv("eval_files/categorical.csv")

### clean up for multiple-choice

In [74]:
mcq_df.head()

,skill,reader,adapter,modelname,timestamp,answer,logits_answer,data_id,dataset,question,context,choices,answer_dataset,logits_answer_0,logits_answer_1
0,multiple-choice,roberta-base,quartz,quant_base,2023-01-26 18:53:22.648838,decrease,"[tensor(, 1.5605,, -1.6844)]",QRQA-10372-1-flip,quartz,If Jim moves some particles of matter farther ...,"When particles of matter are closer together, ...","['decrease', 'increase']",decrease,tensor(,"1.5605,"
1,multiple-choice,roberta-base,quartz,quant_base,2023-01-26 18:53:22.952679,increased,"[tensor(-1.4957,, -1.7537)]",QRQA-10371-4-flip,quartz,Long ago the surface of Venus warmed enough th...,An increase in greenhouse gases leads to great...,"['increased', 'decreased']",decreased,"tensor(-1.4957,",-1.7537)
2,multiple-choice,roberta-base,quartz,quant_base,2023-01-26 18:53:23.288173,less,"[tensor(-0.0722,, 1.9770)]",QRQA-10296-1-flip,quartz,If less waters falls on an area of land it wil...,"As more water covered the land, sand and silt ...","['more', 'less']",less,"tensor(-0.0722,",1.9770)
3,multiple-choice,roberta-base,quartz,quant_base,2023-01-26 18:53:23.601454,open,"[tensor(2.0978,, 3.9687)]",QRQA-10115-3,quartz,Rich applies a solution to the dish that incre...,The increase in turgor pressure of the guard c...,"['close', 'open']",open,"tensor(2.0978,",3.9687)
4,multiple-choice,roberta-base,quartz,quant_base,2023-01-26 18:53:23.916008,older,"[tensor(, 0.6785,, -0.5332)]",QRQA-10082-1,quartz,Simon was digging in his yard and found that t...,"Therefore, deeper rock layers must be older th...","['older', 'younger']",older,tensor(,"0.6785,"


In [83]:
df = pd.DataFrame(columns=[
    "skill", "reader", "adapter", "timestamp", 
    "answer_base", "logits_answer_base",
    "answer_quantized_model", "logits_answer_quantized_model", 
    "answer_onnx_model", "logits_answer_onnx_model",
    "answer_onnx_opt_model", "logits_answer_onnx_opt_model",
    "answer_quant_onnx_model", "logits_answer_quant_onnx_model",
    "answer_quant_onnx_opt_model", "logits_answer_quant_onnx_opt_model",
    "data_id", "dataset", "question", "context", "answer_dataset"
])

In [84]:
skill = mcq_df["skill"].iloc[0]
for reader in mcq_df["reader"].unique():
    # print(reader)
    reader_mcq_df = mcq_df[mcq_df["reader"] == reader]
    # print(len(reader_mcq_df["data_id"]))

    for data_id in reader_mcq_df["data_id"].unique():
        s = reader_mcq_df[reader_mcq_df["data_id"] == data_id].reset_index()

        adapter = s["adapter"].iloc[0]
        data_set_name = adapter
        question = s["question"].iloc[0]
        context = s["context"].iloc[0]
        answer_dataset = s["answer_dataset"].iloc[0]

        interim = {}
        for model in s["modelname"].unique():
            # print(model)
            interim[model] = (
                s[s["modelname"] == model]["answer"].values[0],
                s[s["modelname"] == model]["logits_answer"].values[0]
            )
            
        # print(interim)

        df.loc[len(df)] = [
            skill, reader, adapter,
            pd.Timestamp.now(),

            interim["base"][0], interim["base"][1],
            interim["quant_base"][0], interim["quant_base"][1],
            interim["onnx"][0], interim["onnx"][1],
            interim["onnx_opt"][0], interim["onnx_opt"][1],
            interim["quant_onnx"][0], interim["quant_onnx"][1],
            interim["quant_onnx_opt"][0], interim["quant_onnx_opt"][1],
                                                                                                                            
            data_id, data_set_name, question, context, answer_dataset

        ]

        # break

        

In [85]:
df

,skill,reader,adapter,timestamp,answer_base,logits_answer_base,answer_quantized_model,logits_answer_quantized_model,answer_onnx_model,logits_answer_onnx_model,...,logits_answer_onnx_opt_model,answer_quant_onnx_model,logits_answer_quant_onnx_model,answer_quant_onnx_opt_model,logits_answer_quant_onnx_opt_model,data_id,dataset,question,context,answer_dataset
0,multiple-choice,roberta-base,quartz,2023-02-02 00:36:51.323394,decrease,"tensor([[ 1.5967, -1.6974]])",decrease,"tensor([[ 1.5605, -1.6844]])",decrease,[[-1.1497079 -2.497363 ]],...,[[-1.1497079 -2.497363 ]],decrease,[[-1.0959533 -2.387991 ]],decrease,[[-1.0959533 -2.387991 ]],QRQA-10372-1-flip,quartz,If Jim moves some particles of matter farther ...,"When particles of matter are closer together, ...",decrease
1,multiple-choice,roberta-base,quartz,2023-02-02 00:36:51.332123,increased,"tensor([[-1.4991, -1.7472]])",increased,"tensor([[-1.4957, -1.7537]])",decreased,[[-2.2758887 -1.4322541]],...,[[-2.2758887 -1.4322541]],decreased,[[-2.2649467 -1.5029308]],decreased,[[-2.2649467 -1.5029308]],QRQA-10371-4-flip,quartz,Long ago the surface of Venus warmed enough th...,An increase in greenhouse gases leads to great...,decreased
2,multiple-choice,roberta-base,quartz,2023-02-02 00:36:51.342105,less,"tensor([[-0.0402, 1.9519]])",less,"tensor([[-0.0722, 1.9770]])",less,[[-0.3926896 0.05386359]],...,[[-0.3926896 0.05386359]],less,[[-0.7888222 -0.02593471]],less,[[-0.7888222 -0.02593471]],QRQA-10296-1-flip,quartz,If less waters falls on an area of land it wil...,"As more water covered the land, sand and silt ...",less
3,multiple-choice,roberta-base,quartz,2023-02-02 00:36:51.352452,open,"tensor([[1.9225, 3.9955]])",open,"tensor([[2.0978, 3.9687]])",open,[[3.1936617 3.6193929]],...,[[3.1936617 3.6193929]],open,[[3.340151 3.5526862]],open,[[3.340151 3.5526862]],QRQA-10115-3,quartz,Rich applies a solution to the dish that incre...,The increase in turgor pressure of the guard c...,open
4,multiple-choice,roberta-base,quartz,2023-02-02 00:36:51.362372,older,"tensor([[ 0.6491, -0.4949]])",older,"tensor([[ 0.6785, -0.5332]])",older,[[ 0.73783517 -0.24088499]],...,[[ 0.73783517 -0.24088499]],older,[[ 0.42830256 -0.4287805 ]],older,[[ 0.42830256 -0.4287805 ]],QRQA-10082-1,quartz,Simon was digging in his yard and found that t...,"Therefore, deeper rock layers must be older th...",older
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11797,multiple-choice,bert-base-uncased,cosmos_qa,2023-02-02 00:39:02.067068,They have an illness .,"tensor([[-5.1930, -3.6329, -4.7686, 0.3609]])",They have an illness .,"tensor([[-5.1244, -3.6042, -4.7161, 0.2233]])",They have an illness .,[[-1.221707 -2.1253948 -1.0834084 -0.96413...,...,[[-1.221707 -2.1253948 -1.0834084 -0.96413...,They felt pretty decent by the night .,[[-0.6714138 -0.9072368 -0.8659805 -0.87969...,They felt pretty decent by the night .,[[-0.6714138 -0.9072368 -0.8659805 -0.87969...,3E22YV8GG3KRK7YZLJ4A0IT6QWLNPQ##32UTUBMZ7JB5B2...,cosmos_qa,What 's a possible reason the writer was throw...,Not going in but the antibiotic they put into ...,They have an illness .
11798,multiple-choice,bert-base-uncased,cosmos_qa,2023-02-02 00:39:02.079707,Because it 's in the morning .,"tensor([[-5.5924, -3.8176, -3.6756, -6.5345]])",None of the above choices .,"tensor([[-5.7970, -3.8314, -3.8892, -6.5010]])",Because the writer may give Mike another littl...,[[-1.8153951 -2.4976933 -1.9533283 -2.1874597]],...,[[-1.8153951 -2.4976933 -1.9533283 -2.1874597]],Because the writer may give Mike another littl...,[[-1.1567782 -1.6433551 -1.7078986 -1.2740606]],Because the writer may give Mike another littl...,[[-1.1567782 -1.6433551 -1.7078986 -1.2740606]],3CVBMEMMXDUA458OMZFGVC5P509H7S##32UTUBMZ7JB5B2...,cosmos_qa,What 's a possible reason the writer has an ap...,This morning was my appointment with the psych...,None of the above choices .
11799,multiple-choice,bert-base-uncased,cosmos_qa,2023-02-02 00:39:02.092676,I would n't be a

In [86]:
df.to_csv("eval_files/multiple_choice.csv")

### clean up for span-extraction

In [ ]:
# error in 

In [52]:
span_df.head(10)

,Unnamed: 0,skill,reader,adapter,modelname,timestamp,answer,start,end,data_id,dataset,question,context,answer_dataset,Unnamed: 0.1
0,5,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:00.250959,denver broncos,46,48,56be4db0acb8001400a502ec,drop,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",NaN
1,6,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:00.697431,NaN,57,48,56be4db0acb8001400a502ed,drop,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"{'text': ['Carolina Panthers', 'Carolina Panth...",NaN
2,7,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:01.133585,levi's stadium,78,82,56be4db0acb8001400a502ee,drop,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"{'text': ['Santa Clara, California', ""Levi's S...",NaN
3,8,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:01.567494,denver broncos,43,45,56be4db0acb8001400a502ef,drop,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",NaN
4,9,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:02.010798,gold,118,119,56be4db0acb8001400a502f0,drop,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",NaN
5,10,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:02.442130,NaN,162,110,56be8e613aeaaa14008c90d1,drop,What was the theme of Super Bowl 50?,Super Bowl 50 was an American football game to...,"{'text': ['""golden anniversary""', 'gold-themed...",NaN
6,11,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:02.865466,february,72,73,56be8e613aeaaa14008c90d2,drop,What day was the game played on?,Super Bowl 50 was an American football game to...,"{'text': ['February 7, 2016', 'February 7', 'F...",NaN
7,12,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:03.290591,[CLS],0,1,56be8e613aeaaa14008c90d3,drop,What is the AFC short for?,Super Bowl 50 was an American football game to...,"{'text': ['American Football Conference', 'Ame...",NaN
8,13,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:03.718973,NaN,162,110,56bea9923aeaaa14008c91b9,drop,What was the theme of Super Bowl 50?,Super Bowl 50 was an American football game to...,"{'text': ['""golden anniversary""', 'gold-themed...",NaN
9,14,span-extraction,bert-base-uncased,drop,onnx,2023-02-08 17:44:04.137703,[CLS],0,1,56bea9923aeaaa14008c91ba,drop,What does AFC stand for?,Super Bowl 50 was an American football game to...,"{'text': ['American Football Conference', 'Ame...",NaN


In [53]:
span_df.dataset = "squad"

In [19]:
span_df.to_csv("eval_files/span_scores.csv")
span_df.to_excel("eval_files/span_scores.xlsx")